# Elastic Search Handbook

### Ideas

1. Use this notebook to get polygons correct for California Bucket

## lsaa-staging-cog

2. grab the latest map.geojson - for example map(2).geojson by create date - python code

## elastic search lesson 1

#### Two Fake Path Rows

save as pr1.geojson and pr2.geojson over fresno


[link to geojson](http://geojson.io/#map=10/36.8208/-119.3019)




In [1]:
#!  http://geojson.io/#map=10/36.8208/-119.3019

print("Now grab a box - ~/Downloads/map.geojson")



Now grab a box - ~/Downloads/map.geojson


In [2]:
!scp sleepy:~/Downloads/fake_path_row.geojson .
! cat fake_path_row.geojson | python3 -m json.tool

fake_path_row.geojson                         100%  595   202.1KB/s   00:00    
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            -120.157470703125,
                            36.13343831245866
                        ],
                        [
                            -118.9434814453125,
                            36.13343831245866
                        ],
                        [
                            -118.9434814453125,
                            37.28279464911045
                        ],
                        [
                            -120.157470703125,
                            37.28279464911045
                        ],
                        [
                            -120.157470703125,
                     

# path row to elastic search

1. 


### Version

[what version am i running click HERE](http://elastic:9200)


### References

https://blog.benjie.me/geospatial-queries-for-elasticsearch/

[Getting started with Elasticsearch in Python](https://towardsdatascience.com/getting-started-with-elasticsearch-in-python-c3598e718380)

https://www.elastic.co/assets/bltf3aec2bf05b22820/elasticon18-state-of-geo-in-elasticsearch.pdf


### dont forget the awesome google json plugin with the TREE VIEW

In [3]:
# %%bash
# curl -vX POST elastic:9200/rawgeo/FeatureCollection -d @fake_path_row.geojson \
# --header "Content-Type: application/json"


## Oops - forgot to create type and it looks like I will need two files

1. create fake_PR_1.geojson
2. fake_PR_2.geojson

[link to geojson](http://geojson.io/#map=10/36.8208/-119.3019)

In [4]:
#!scp sleepy:~/Downloads/\*.geojson .
#!ls

In [5]:
#! mv map.geojson fake_PR_2.geojson
#! mv "map(1).geojson" fake_PR_1.geojson

In [6]:
import logging
from elasticsearch import Elasticsearch
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'elastic', 'port': 9200}])
    if _es.ping():
        print('Yay Connect')
    else:
        print('Awww it could not connect!')
    return _es

if __name__ == '__main__':
  logging.basicConfig(level=logging.ERROR)

In [7]:
def create_index(es_object, index_name='georaw'):
    created = False
    # index settings
    # index = georaw, type/group = FeatureCollection
    #       field name/property name = feastues - cause that is what geojson calls them!
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "record": {
                "dynamic": "strict",
                "properties": {
                    "footprint": {
                        "type": "geo_shape"
                    },
                    "bogus": {
                        "type": "text"
                    }
                }
            }
        }
    }

    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [8]:
es_conn = connect_elasticsearch()
# delete any old indexes - similar to clearing the postgres db
es_conn.indices.delete(index='georaw', ignore=[400, 404])

create_index(es_conn, index_name='georaw')


Yay Connect
Created Index


True

In [9]:
# %%bash
# curl -vX POST elastic:9200/rawgeo/FeatureCollection -d @fake_PR_1.geojson \
# --header "Content-Type: application/json"

# Now add 1 fake Path Row
# not quite right - TRY AGAIN!

In [10]:
! echo date

date


In [11]:
import json
def create_es_record_from_geojson(geo_file):
    print(geo_file)
    with open(geo_file, 'r+') as f:
        data = json.load(f)
        print(data)
        print("###"*23)
        
        #print(json.dump(data, f, indent=4))
        #f.seek(0)        # <--- should reset file position to the beginning.
        #json.dump(data, f, indent=4)
        # f.truncate()     # remove remaining part
    return (data)


In [12]:
! cat fake_PR_1.geojson

{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-120.157470703125,36.13343831245866],[-118.9434814453125,36.13343831245866],[-118.9434814453125,37.28279464911045],[-120.157470703125,37.28279464911045],[-120.157470703125,36.13343831245866]]]}}]}

In [13]:
es_record = create_es_record_from_geojson("fake_PR_1.geojson")

fake_PR_1.geojson
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-120.157470703125, 36.13343831245866], [-118.9434814453125, 36.13343831245866], [-118.9434814453125, 37.28279464911045], [-120.157470703125, 37.28279464911045], [-120.157470703125, 36.13343831245866]]]}}]}
#####################################################################


In [14]:
print(es_record)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-120.157470703125, 36.13343831245866], [-118.9434814453125, 36.13343831245866], [-118.9434814453125, 37.28279464911045], [-120.157470703125, 37.28279464911045], [-120.157470703125, 36.13343831245866]]]}}]}


In [15]:
! dir(es_record)

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


In [16]:
import pygeoj
def geo_walk(geo_file):
    testfile = pygeoj.load(geo_file)
    for feature in testfile:
        print (feature.geometry.type)
        print (feature.geometry.coordinates)
        foot = {
            "type": feature.geometry.type,
            "coordinates": feature.geometry.coordinates
        }
        return (foot)

In [17]:
#! pip3 install pygeoj

In [18]:
poly = geo_walk("fake_PR_1.geojson")

Polygon
[[[-120.157470703125, 36.13343831245866], [-118.9434814453125, 36.13343831245866], [-118.9434814453125, 37.28279464911045], [-120.157470703125, 37.28279464911045], [-120.157470703125, 36.13343831245866]]]


In [19]:
poly

{'type': 'Polygon',
 'coordinates': [[[-120.157470703125, 36.13343831245866],
   [-118.9434814453125, 36.13343831245866],
   [-118.9434814453125, 37.28279464911045],
   [-120.157470703125, 37.28279464911045],
   [-120.157470703125, 36.13343831245866]]]}

In [20]:
def elastic_flatten_doc(poly):
    elastic_doc = {
                    'bogus': 'bogusTestTony',
                    'footprint': poly
                }
    return elastic_doc

In [21]:
e_doc = elastic_flatten_doc(poly)
print(e_doc)

{'bogus': 'bogusTestTony', 'footprint': {'type': 'Polygon', 'coordinates': [[[-120.157470703125, 36.13343831245866], [-118.9434814453125, 36.13343831245866], [-118.9434814453125, 37.28279464911045], [-120.157470703125, 37.28279464911045], [-120.157470703125, 36.13343831245866]]]}}


In [22]:
def store_record(elastic_object, index_name, record):
    try:
        outcome = elastic_object.index(index=index_name, doc_type='record', body=record)
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))


In [23]:
import pprint
elastic_json_record = json.dumps(e_doc)
print("###"*30)
pprint.pprint(elastic_json_record)
store_record(es_conn,'georaw',elastic_json_record)


##########################################################################################
('{"bogus": "bogusTestTony", "footprint": {"type": "Polygon", "coordinates": '
 '[[[-120.157470703125, 36.13343831245866], [-118.9434814453125, '
 '36.13343831245866], [-118.9434814453125, 37.28279464911045], '
 '[-120.157470703125, 37.28279464911045], [-120.157470703125, '
 '36.13343831245866]]]}}')


# now do a search with geojson.io

[link to geojson](http://geojson.io/#map=10/36.8208/-119.3019)

In [24]:
#!scp sleepy:./Downloads/\*.geojson .

In [25]:
#!ls

In [26]:
#! cp 'map(2).geojson' search_input.geojson

In [27]:
poly_of_interest =  geo_walk("search_input.geojson")

Polygon
[[[-119.37744140625, 36.81368259124586], [-119.234619140625, 36.81368259124586], [-119.234619140625, 36.92793899776678], [-119.37744140625, 36.92793899776678], [-119.37744140625, 36.81368259124586]]]


In [28]:
# hi

In [29]:
def return_geo_query_body(poly):

    geo_filter_body = {"query": {
    "bool": {
        "must": {
                "match_all": {}
            },
      "filter": [ {
      "geo_shape": {
                    "footprint": {
                        "shape": poly
                    }
                }
      }
      ]
    }
    }}

    return geo_filter_body


In [30]:
my_query = return_geo_query_body(poly_of_interest)

In [31]:
my_query

{'query': {'bool': {'must': {'match_all': {}},
   'filter': [{'geo_shape': {'footprint': {'shape': {'type': 'Polygon',
        'coordinates': [[[-119.37744140625, 36.81368259124586],
          [-119.234619140625, 36.81368259124586],
          [-119.234619140625, 36.92793899776678],
          [-119.37744140625, 36.92793899776678],
          [-119.37744140625, 36.81368259124586]]]}}}}]}}}

this does not work yet
```
GET /datacube/_search
"query": {
        "bool": {
            "must": {
                "match_all": {}
            },
            "filter": {
                "geo_shape": {
                    "footprint": {
                        "shape": {
                            "type": "Polygon",
                            "coordinates": 
                            [[
          [-119.37744140625,36.81368259124586],
          [-119.234619140625, 36.81368259124586],
          [-119.234619140625, 36.92793899776678],
          [-119.37744140625, 36.92793899776678],
          [-119.37744140625, 36.81368259124586]
          ]]
                        }
                    }
                }
            }
        }
    }
```

```json
GET /datacube/_search
"query": {
        "bool": {
            "must": {
                "match_all": {}
            },
            "filter": {
                "geo_shape": {
                    "footprint": {
                        "shape": {
                            "type": "Polygon",
                            "coordinates": 
                            [[
          [-119.37744140625,36.81368259124586],
          [-119.234619140625, 36.81368259124586],
          [-119.234619140625, 36.92793899776678],
          [-119.37744140625, 36.92793899776678],
          [-119.37744140625, 36.81368259124586]
          ]]
                        }
                    }
                }
            }
        }
    }
```

In [32]:
%%bash
curl -vX GET elastic:9200/georaw/_mapping | python3 -m json.tool

{
    "georaw": {
        "mappings": {
            "record": {
                "dynamic": "strict",
                "properties": {
                    "bogus": {
                        "type": "text"
                    },
                    "footprint": {
                        "type": "geo_shape"
                    }
                }
            }
        }
    }
}


Note: Unnecessary use of -X or --request, GET is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 192.168.1.225...
* TCP_NODELAY set
* Connected to elastic (192.168.1.225) port 9200 (#0)
> GET /georaw/_mapping HTTP/1.1
> Host: elastic:9200
> User-Agent: curl/7.58.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< content-type: application/json; charset=UTF-8
< content-length: 127
< 
{ [127 bytes data]
100   127  100   127    0     0   7470      0 --:--:-- --:--:-- --:--:--  7937
* Connection #0 to host elastic left intact


In [33]:
%%bash
curl -vX GET elastic:9200/datacube/_mapping | python3 -m json.tool

{
    "datacube": {
        "mappings": {
            "rwanda": {
                "dynamic": "strict",
                "properties": {
                    "blue": {
                        "type": "text"
                    },
                    "creation_dt": {
                        "type": "date",
                        "format": "yyyy-MM-dd||yyyy"
                    },
                    "footprint": {
                        "type": "geo_shape"
                    },
                    "green": {
                        "type": "text"
                    },
                    "nir": {
                        "type": "text"
                    },
                    "pixel_qa": {
                        "type": "text"
                    },
                    "processing_level": {
                        "type": "text"
                    },
                    "red": {
                        "type": "text"
                    },
                    "ul": {
               

Note: Unnecessary use of -X or --request, GET is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 192.168.1.225...
* TCP_NODELAY set
* Connected to elastic (192.168.1.225) port 9200 (#0)
> GET /datacube/_mapping HTTP/1.1
> Host: elastic:9200
> User-Agent: curl/7.58.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< content-type: application/json; charset=UTF-8
< content-length: 342
< 
{ [342 bytes data]
100   342  100   342    0     0  19000      0 --:--:-- --:--:-- --:--:-- 20117
* Connection #0 to host elastic left intact


In [34]:
%%bash
curl --header 'Content-Type: application/json' -XPOST 'http://elastic:9200/georaw/_search?size=400' -d \
'{"query": {"bool": {"must": {"match_all": {}},
   "filter": [{"geo_shape": {"footprint": {"shape": {"type": "Polygon",
        "coordinates": [[[-119.37744140625, 36.81368259124586],
          [-119.234619140625, 36.81368259124586],
          [-119.234619140625, 36.92793899776678],
          [-119.37744140625, 36.92793899776678],
          [-119.37744140625, 36.81368259124586]]]}}}}]}}}'

{"took":1,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":0,"max_score":null,"hits":[]}}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   523  100   134  100   389  11166  32416 --:--:-- --:--:-- --:--:-- 43583


In [35]:
%%bash
echo '{"query": {"bool": {"must": {"match_all": {}},
   "filter": [{"geo_shape": {"footprint": {"shape": {"type": "Polygon",
        "coordinates": [[[-119.37744140625, 36.81368259124586],
          [-119.234619140625, 36.81368259124586],
          [-119.234619140625, 36.92793899776678],
          [-119.37744140625, 36.92793899776678],
          [-119.37744140625, 36.81368259124586]]]}}}}]}}}' |python3 -m json.tool

{
    "query": {
        "bool": {
            "must": {
                "match_all": {}
            },
            "filter": [
                {
                    "geo_shape": {
                        "footprint": {
                            "shape": {
                                "type": "Polygon",
                                "coordinates": [
                                    [
                                        [
                                            -119.37744140625,
                                            36.81368259124586
                                        ],
                                        [
                                            -119.234619140625,
                                            36.81368259124586
                                        ],
                                        [
                                            -119.234619140625,
                                            36.92793899776678
                 

# Now lets do this with code python

In [36]:
def _local_return_elastic_hits(es_conn, the_query):
    
   
    response = es_conn.search( index="georaw",size=1400, body=the_query)

    HIT_LIST = response['hits']['hits']
    return HIT_LIST


In [37]:
hits = _local_return_elastic_hits(es_conn, my_query)

In [38]:
pprint.pprint (hits)

[{'_id': '1W46zWgBYzFhnv3daI4L',
  '_index': 'georaw',
  '_score': 1.0,
  '_source': {'bogus': 'bogusTestTony',
              'footprint': {'coordinates': [[[-120.157470703125,
                                              36.13343831245866],
                                             [-118.9434814453125,
                                              36.13343831245866],
                                             [-118.9434814453125,
                                              37.28279464911045],
                                             [-120.157470703125,
                                              37.28279464911045],
                                             [-120.157470703125,
                                              36.13343831245866]]],
                            'type': 'Polygon'}},
  '_type': 'record'}]
